In [1]:
import glob
import os
from subprocess import check_output
import argparse
import re


In [2]:
def make_file(sub,main_dict, task, deriv_dir, fsf_template,sessions):
    print("making file for subject ", sub)
    for sess_id in sessions:


            if task == "resting":
                # case - no runs, only single task (i.e. resting)
                pass

            else:
                for key in main_dict[sub]:
                    if key != "ANAT":
                        run = key
                        print(run)
                        outpath = os.path.join(deriv_dir, sub, 'ses-1/func', 'Analysis', "feat1")
                        if not os.path.exists(outpath):
                            os.makedirs(outpath)


                        with open(fsf_template, 'r') as infile:
                            print("Opening template file {}".format(fsf_template))
                            tempfsf = infile.read()

                            #  fill in tempfsf file with parameters
                            tempfsf = tempfsf.replace("OUTDIR", main_dict[sub][run]["OUTPUT"])
                            tempfsf = tempfsf.replace("FUNCTIONAL", main_dict[sub][run]["FUNC"])
                            tempfsf = tempfsf.replace("TR", main_dict[sub][run]['TR'])
                            tempfsf = tempfsf.replace("CONFOUND", main_dict[sub][run]['CONFOUND'])
                            tempfsf = tempfsf.replace("VOL", main_dict[sub][run]['VOL'])


                            # loop through keys in dict to find EVs and MOCOs
                            for key in main_dict[sub][run]:


                                # Fill in EVS
                                if re.match(r'EV', key):
                                    ev_name = key.split("_")[1]
                                    ev = main_dict[sub][run][key]
                                    tempfsf = tempfsf.replace(ev_name+"_file", ev)
                                    #print(ev_name, "\n", ev)

                                if re.match(r'moco', key):
                                    moco_file = main_dict[sub][run][key]
                                    tempfsf = tempfsf.replace(key+"_file", moco_file)
                                    #print(moco_file)

                            fsf_outfile = '%s_run-%s_fsl.fsf' % (task, run)
                            with open(os.path.join(outpath, fsf_outfile), 'w') as outfile: #os.path.join(outpath,
                                outfile.write(tempfsf)
                            outfile.close()
                        infile.close()


In [3]:




def fill_dict(sub, main_dict, task, deriv_path, sessions,evs, all_runs):
    sub_path = os.path.join(deriv_path, sub)
    #print("SUBJECT: %s \t TASK: %s \nPATH: %s"% (sub, task, sub_path))


# only specified sessions
    for sess_id in sessions:

        if task == 'resting':
            # case for no runs, only task (i.e. resting)
            pass
        else:
        # 2 cases: individual/given runs or all runs found

            # case 1: if flag false, grab all available runs found
            if all_runs == True:
                funcs_found = glob.glob(os.path.join(deriv_path, sub, 'ses-1/func',
                                             "%s_%s_task-%s_run-*preproc*brain.nii.gz" % (sub, sess_id, task)))
                runs=[x.split("/")[-1].split("_")[3].split("-")[1] for x in funcs_found]
                for run in runs:
                    main_dict[sub][run] = {}
                #print("Dictionary initialized as: {}".format(main_dict[sub]))

                for func in funcs_found:
                    x = int(run)
                    run=func.split("/")[-1].split("_")[3].split("-")[1]

                    # SET OUTPUT PATH FOR FEAT DIRECTORY
                    output_path=os.path.join(sub_path, 'ses-1/func',
                                             'Analysis', 'feat1', '%s_run-%s_fsl' %(task, run))


                    # SET CONFOUND
                    # sub-004_ses-1_task-resting_space-MNI152NLin2009cAsym_desc-preproc_confound.txt
                    # %s_ses-%s_task-%s_run-%s_confound.txt
                    confound = os.path.join(deriv_path, sub, 'ses-1/func', 'motion_assessment',
                                     '%s_%s_task-%s_run-%s_space-MNI152NLin2009cAsym_desc-preproc_confound.txt' % (sub, sess_id, task, run))

                    # SET ANAT
                    #anat = os.path.join(deriv_path, sub, 'ses-1/anat', 'highres.nii.gz')





                    # FILL DICTIONARY
                    #main_dict[sub]['ANAT'] = anat
                    main_dict[sub][run]['OUTPUT'] = output_path
                    scan = func.split(".")[0]
                    main_dict[sub][run]['FUNC'] = scan
                    vol = check_output(['fslnvols', scan])
                    vol = vol.decode('utf-8')
                    vol = vol.strip('\n')
                    main_dict[sub][run]['VOL'] = vol
                    main_dict[sub][run]['CONFOUND'] = confound
                    
                    
                    
                    # TRS FROM NIFTI -- this value will always be 2, therefore we only run the check once
                    trs = check_output(['fslval', '%s' % (scan), 'pixdim4', scan])
                    trs = trs.decode('utf-8')
                    trs = trs.strip('\n')
                    # print("TRs: ", trs)

                    main_dict[sub][run]['TR'] = trs


                    # SET MOTION PARAMETERS
                    for i in range(6):
                        motcor = os.path.join(sub_path, 'ses-1/func', 'motion_assessment', 'motion_parameters',
                                          '%s_%s_task-%s_run-%s_moco%s.txt' % (sub, sess_id, task, run, i))
                        main_dict[sub][run]['moco%i' % i] = motcor


                    # SET EVS
                    # Loop through the given EVs and add the corresponding file to the dictionary

                    ctr = 0
                    for ev_name in evs:
                        # print(item)
                        ctr = ctr + 1

                        ev = os.path.join(sub_path, "ses-1/func", "onsets", '%s_%s_task-%s-run-%s.tsv' % (sub,sess_id ev_name, run))
                        #print(ev)
                        # print("EV: ", ev)
                        main_dict[sub][run]['EV_%s' % ev_name] = ev


            else:
                ## go through runs given in arguments

                pass





In [12]:
def main():
    #set_paths()
    # removed path function for now
    print("Starting program....")
    deriv_dir = "/projects/niblab/bids_projects/Experiments/bbx/bids/derivatives"
    main_dict = {}
    subject_folders = sorted(glob.glob(os.path.join(deriv_dir, 'sub-*')))
    run_bash =False
    write_file = False
    
    if write_file == True:
        ## case: Get all subjects available --add flag for individual subjects or passed list option
        for sub_path in subject_folders:

            # set variables
            sub = sub_path.split("/")[-1]
            task = "training"
            sessions = ["ses-1"]
            evs = ['SSB', 'USB', 'H2O', 'H2Ocue', 'SSBcue', 'USBcue', 'rinse']

            all_runs = True
            fsf_template = os.path.join('/projects/niblab/bids_projects/Experiments/bbx/bids/code/design_files','design1.fsf')


            #set_dict(sub)
            main_dict[sub] = {}

            # set up dict for runs IF flag is passed
            # def fill_dict(sub, main_dict, task, deriv_path, sessions,evs, all_runs)
            fill_dict(sub, main_dict, task, deriv_dir, sessions, evs, all_runs)

            #print(">>>> ",main_dict[sub])

            #def make_file(sub,main_dict, run, task, deriv_dir, fsf_template)
            make_file(sub, main_dict, task, deriv_dir, fsf_template,sessions)
    
    subject_set = subject_folders[4:]
    subject_set = [x.split("/")[-1].split("-")[1].lstrip("0") for x in subject_set]

    if run_bash == True:
        bash_file = os.path.join('/projects/niblab/bids_projects/Experiments/bbx/bids/code', 'feat1_fsl.job')
        
        for sub_num in subject_set:
            shell_cmd = "sbatch --array={}%1 {}".format(sub_num, bash_file)
            os.system(shell_cmd)
            print(shell_cmd)
    

In [13]:
main()

Starting program....


In [9]:
!squeue -u nbytes

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
         2527428_5     batch BBX_FEAT   nbytes  R       0:00      1 compute-8-5
         2527429_6     batch BBX_FEAT   nbytes  R       0:00      1 compute-8-5
         2527430_7     batch BBX_FEAT   nbytes  R       0:00      1 compute-8-5
         2527431_8     batch BBX_FEAT   nbytes  R       0:00      1 compute-8-5
         2527432_9     batch BBX_FEAT   nbytes  R       0:00      1 compute-8-5
        2527433_10     batch BBX_FEAT   nbytes  R       0:00      1 compute-8-5
        2527434_11     batch BBX_FEAT   nbytes  R       0:00      1 compute-8-5
        2527435_12     batch BBX_FEAT   nbytes  R       0:00      1 compute-8-0
        2527436_13     batch BBX_FEAT   nbytes  R       0:00      1 compute-8-0
        2527437_14     batch BBX_FEAT   nbytes  R       0:00      1 compute-8-0
        2527438_15     batch BBX_FEAT   nbytes  R       0:00      1 compute-8-0
        2527439_16     batch BBX_FE